* Использую данные по airdrop EIGEN отсюда https://dune.com/p2p_org/eigenlayer-airdrops
* Так как есть информация только о кошельках получивших airdrop то модель должна предсказать 0 (не sybil)
* Собираем фичи с помощью Moralis API

In [62]:
import json

from tqdm.auto import tqdm
import pandas as pd

from sklearn.metrics import accuracy_score, classification_report

In [63]:
with open('data.txt') as f:
    valid_wallets = json.load(f)

In [64]:
import sys
sys.path.append('..')

from src.moralis_extractor import MoralisClient
from src.feature_extractor import calculate_features_moralis

In [ ]:
import yaml

with open('../config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

client = MoralisClient(config['moralis_api_key'], timeout=100)

In [ ]:
all_features = {}

In [12]:
for wallet in tqdm(valid_wallets):
    if wallet in all_features:
        continue
    try:
        normal_txs = client.fetch_normal_transactions(wallet)
        token_txs = client.fetch_token_transfers(wallet)
        features = calculate_features_moralis(normal_txs, token_txs, wallet)
    except Exception as e:
        print(f"Error fetching data for wallet {wallet}: {e}")
        continue
    all_features[wallet] = features

 54%|█████▍    | 163/301 [00:02<00:01, 72.42it/s]

Error fetching data for wallet 0x7c829a56e5a0c6625bcc39b774e18e745a345d67: 'NoneType' object has no attribute 'lower'


 58%|█████▊    | 176/301 [00:02<00:01, 70.39it/s]

Error fetching data for wallet 0xa089.eth: 400 Client Error: Bad Request for url: https://deep-index.moralis.io/api/v2/0xa089.eth?chain=eth


100%|██████████| 301/301 [04:20<00:00,  1.15it/s]


In [13]:
with open('features.json', 'w') as f:
    json.dump(all_features, f, indent=4)

In [14]:
len(all_features)

299

In [65]:
df = pd.DataFrame.from_dict(all_features, orient='index')
target = [0] * len(df)

In [66]:
import pickle

with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [67]:
type(model)

sklearn.ensemble._forest.RandomForestClassifier

In [68]:
y_pred = model.predict(df)
y_prob = model.predict_proba(df)[:, 1]

print("Accuracy:", accuracy_score(target, y_pred))
print(classification_report(target, y_pred, zero_division=0))

Accuracy: 0.7357859531772575
              precision    recall  f1-score   support

           0       1.00      0.74      0.85       299
           1       0.00      0.00      0.00         0

    accuracy                           0.74       299
   macro avg       0.50      0.37      0.42       299
weighted avg       1.00      0.74      0.85       299

